In [ ]:
# Core Libraries
import pandas as pd
import numpy as np
import torch
import psutil
import cpuinfo

# Experiment Tracking
import wandb

# Sklearn Metrics
from sklearn.metrics import (
    accuracy_score, 
    precision_recall_fscore_support, 
    matthews_corrcoef, 
    confusion_matrix
)

# Hugging Face Datasets
from datasets import Dataset, load_from_disk

# Hugging Face Transformers
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)

# PEFT (Parameter Efficient Fine-Tuning)
from peft import (
    LoraConfig, 
    get_peft_model, 
    TaskType,
    PeftModel
)

In [ ]:
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("wandb_api")
    wandb.login(key=api_key)
    anony = None
except:
    anony = "must"
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

# GPU & CPU Specs

In [ ]:
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))

In [ ]:
# Get CPU model and cores
cpu_info = cpuinfo.get_cpu_info()
print(f"CPU Model: {cpu_info['brand_raw']}")
print(f"Cores: {psutil.cpu_count(logical=False)} Physical / {psutil.cpu_count(logical=True)} Logical")

# Get clock speed
print(f"Max Frequency: {psutil.cpu_freq().max:.2f} MHz")

# Check RAM
print(f"RAM: {psutil.virtual_memory().total / (1024**3):.1f} GB")

# Dataloading

In [ ]:
# Load from read-only input
original_dataset = load_from_disk("/kaggle/input/merged/tokenized_merged_wo_emojis")

working_dir = "/kaggle/working/merged/tokenized_merged_wo_emojis"
# Save a copy to a writable location
original_dataset.save_to_disk(working_dir)

# Load the writable copy
tokenized_dataset = load_from_disk(working_dir, keep_in_memory=True)

In [ ]:
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"], device="cuda")

In [ ]:
print(tokenized_dataset["train"].column_names)

# Training

In [ ]:
def compute_metrics(pred):
    logits, labels = pred.predictions, pred.label_ids
    
    # Optional sanity check: make sure logits are not probabilities
    if logits.shape[1] == 2:  # Binary classification logits
        probs = torch.softmax(torch.tensor(logits), dim=1)[:, 1].numpy()
    else:
        raise ValueError("Unexpected logits shape: make sure model outputs raw logits with 2 classes.")
    
    preds = (probs > 0.65).astype(int)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    precision_per_class, recall_per_class, f1_per_class, _ = precision_recall_fscore_support(labels, preds, average=None, zero_division=0)

    acc = accuracy_score(labels, preds)
    mcc = matthews_corrcoef(labels, preds)

    wandb.log({
        "confusion_matrix": wandb.plot.confusion_matrix(
            probs=None,
            y_true=labels,
            preds=preds,
            class_names=["Non-Hate", "Hate"]
        ),
        "threshold_used": 0.65  # <-- Log the threshold too!
    })

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'matthews_corrcoef': mcc,
        'precision_class_0': precision_per_class[0],
        'precision_class_1': precision_per_class[1],
        'recall_class_0': recall_per_class[0],
        'recall_class_1': recall_per_class[1],
        'f1_class_0': f1_per_class[0],
        'f1_class_1': f1_per_class[1],
    }

In [ ]:
# ============================
# 📦 Dataset & Batching Config
# ============================
data_config = {
    "per_device_train_batch_size": 512,
    "per_device_eval_batch_size": 256,
    "dataloader_pin_memory": False,
    "label_names": ["labels"]
}

# ========================
# ⚙️ Training Runtime Setup
# ========================
runtime_config = {
    "num_train_epochs": 1,
    "gradient_accumulation_steps": 2,
    "fp16": True,
    "gradient_checkpointing": True
}

# =============================
# 🔧 Optimization Configuration
# =============================
optimizer_config = {
    "optim": "adamw_torch_fused",
    "lr_scheduler_type": "cosine",
    "learning_rate": 2e-3,
    "max_grad_norm": 1.0,
    "warmup_ratio": 0.1,
    "weight_decay": 0.01
}

# ==============================
# 📊 Logging & Saving Strategy
# ==============================
logging_config = {
    "report_to": "wandb",
    "logging_steps": 200,
    "eval_strategy": "steps",
    "eval_steps": 200,
    "save_strategy": "steps",
    "save_steps": 200,
    "load_best_model_at_end": True,
    "metric_for_best_model": "matthews_corrcoef",
    "greater_is_better": True
}

# ===============================
# 🧬 Final Training Configuration
# ===============================
training_kwargs = {
    **data_config,
    **runtime_config,
    **optimizer_config,
    **logging_config
}

In [ ]:
model_name = 'vinai/bertweet-base'

model_kwargs = {
    "num_labels": 2,
    "device_map": "cuda",
    "low_cpu_mem_usage": True,
    # "quantization_config": bnb_config
}

In [ ]:
# Init W&B run
wandb.init(project="bertweet-lora-bayes-v2", name="final_run_mcc_0.65_v3")

# Best config
best_config = {
    "r": 16,
    "alpha": 12,
    "target_modules": ["query", "key", "value", "attention.output.dense"]
}

# Setup LoRA
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=best_config["r"],
    lora_alpha=best_config["alpha"],
    lora_dropout=0.1,
    bias="all",
    target_modules=best_config["target_modules"],
    modules_to_save=["classifier"]
)

# Load and prepare model
model = AutoModelForSequenceClassification.from_pretrained(model_name, **model_kwargs)
model = get_peft_model(model, peft_config)
model.config.id2label = {0: "Non-Hate", 1: "Hate"}
model.config.label2id = {"Non-Hate": 0, "Hate": 1}
model.print_trainable_parameters()

# Training arguments
training_args = TrainingArguments(**training_kwargs)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train and Evaluate
trainer.train()

model.save_model("./final_model")
artifact = wandb.Artifact("final_model", type="model")
artifact.add_dir("./final_model")
wandb.log_artifact(artifact)

# Post-training test evaluation (at threshold 0.65)
test_preds = trainer.predict(tokenized_dataset["test"])
test_probs = torch.softmax(torch.tensor(test_preds.predictions), dim=1)[:, 1].numpy()
test_labels = test_preds.label_ids
final_preds = (test_probs > 0.65).astype(int)

final_mcc = matthews_corrcoef(test_labels, final_preds)
final_acc = accuracy_score(test_labels, final_preds)

wandb.log({
    "test/mcc_thresh_0.65": final_mcc,
    "test/accuracy_thresh_0.65": final_acc
})


In [ ]:
print(model)

In [ ]:
# Assume model is a LoRA-wrapped PEFT model
merged_model = model.merge_and_unload()  # Merges LoRA into base
# print(merged_model)

# # Then save the merged model
merged_model.save_pretrained("final_full_model")
# tokenizer.save_pretrained("final_full_model")

artifact = wandb.Artifact("final_model", type="model")
artifact.add_dir("./final_full_model")
wandb.log_artifact(artifact)